# 探索性数据分析(EDA)

## 查看数据基本信息

In [14]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [15]:
import xgboost as xgb

In [16]:
data_fram = pd.read_csv('NetworksOD_180805.csv')
# 查看数据前 5 行
data_fram.head()

,distances,CR_staPath,CR_expPath,CR_godPath,CR_grdyPath,averObjValue_staPath,averObjValue_expPath,averObjValue_godPath,averObjValue_grdyPath,VAR_staPath,...,noweight_avg_degree,strSpd_edges_avg_BC,strSpd_edges_std_BC,strSpd_avg_CC,strSpd_std_CC,Outputs14,Outputs7,Outputs3_1,Outputs3_2,Outputs3_3
0,0.089257,38,36,33,34,1002.583614,950.442450,1003.926941,1014.325558,5587.262540,...,2.156250,0.085249,0.058916,0.093363,0.030391,13,6,-1,0,-1
1,0.053294,14,11,11,10,691.118469,588.300449,746.190531,742.329823,5121.227202,...,2.142857,0.065123,0.033024,0.134397,0.048843,2,2,1,1,-1
2,0.080272,22,18,18,17,981.840365,775.495436,780.134362,844.508519,3770.183203,...,2.107143,0.030206,0.014002,0.053449,0.019681,14,7,-1,-1,0
3,0.063568,11,11,11,11,704.890220,706.739953,732.371605,699.944470,4303.335085,...,2.000000,0.098954,0.044872,0.128724,0.052665,14,7,-1,-1,0
4,0.047176,18,18,17,17,644.648402,630.084259,637.053175,641.776450,3198.230906,...,2.064516,0.095766,0.056636,0.131816,0.064134,14,7,-1,-1,0


In [17]:
# 查看每个类别数量
data_fram['Outputs14'].value_counts()

14    25834
13    20955
12    10074
7      8749
6      8065
1      8028
4      3619
10     3278
2      2721
3      2491
5      2279
8      2237
11     2068
9      1521
Name: Outputs14, dtype: int64

## 删除无用信息

In [18]:
# 删除多余分类标准
data_fram.drop(['Outputs7','Outputs3_1','Outputs3_2','Outputs3_3'],axis=1,inplace=True)

## 对数据进行分层采样

In [19]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2018)

for train_index, test_index in split.split(data_fram, data_fram['Outputs14']):
    train_data = data_fram.loc[train_index]
    test_data = data_fram.loc[test_index]

In [20]:
# 转换数据类型
train_data_numpy_ndarray = train_data.values
test_data_numpy_ndarray = test_data.values

In [21]:
X_train = train_data_numpy_ndarray[:,0:126].astype(float)
Y_train = train_data_numpy_ndarray[:,126].astype(int)
X_test = test_data_numpy_ndarray[:,0:126].astype(float)
Y_test = test_data_numpy_ndarray[:,126].astype(int)

# 实验

## 实验一

In [8]:
model = XGBClassifier() 

In [9]:
model.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [10]:
y_pred = model.predict(X_test)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [11]:
predictions = [round(value) for value in y_pred]

In [13]:
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 34.12%


## 实验二

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import *

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:

xgb_model = xgb.XGBClassifier()

#brute force scan for all parameters, here are the tricks
#usually max_depth is 6,7,8
#learning rate is around 0.05, but small changes may make big diff
#tuning min_child_weight subsample colsample_bytree can have 
#much fun of fighting against overfit 
#n_estimators is how many round of boosting
#finally, ensemble xgboost with multiple seeds may reduce variance
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective': 'multi:softmax', #多分类的问题
              'num_class':14, # 类别数，与 multisoftmax 并用
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [100], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [2018],
             'eval_metric':'merror'}

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(Y_train, n_folds=3, shuffle=True), 
                   scoring='roc_auc',
                   verbose=2, refit=True)


ValueError: Parameter values for parameter (objective) need to be a sequence(but not a string) or np.ndarray.

In [ ]:
clf.fit(train[features], train["QuoteConversion_Flag"])